# More mappers!

# Mapping here simply means converting an existing value that's already in the data into another value using a dictionary as a lookup.

# Examples of this could be mapping similar AIS Codes to standard group names, or adding a dynamic string descriptor based on the speed of the agent (e.g. slow, medium, fast). This is where you can get creative.

# First, you can map metadata (static data) for the agents in your `Dataset`

In [1]:
import trackio as tio

In [2]:
#define the data_path
data_path = 'example0'

#read the existing Dataset
ds = tio.read(data_path=data_path)

#check the metadata available
ds

Type:
    <class 'trackio.Dataset.Dataset'>
Status:
    0 Unprocessed CSV Files
    0 Processed CSV Files
    0 Unsplit Agent Files
    172 Split Agent Files
Static Data Fields:
    ['AISCode', 'Agent ID', 'Length', 'MMSI', 'Name', 'Width']
Dynamic Data Fields:
    ['Coursing', 'Draft', 'Speed', 'Status', 'Time', 'X', 'Y']
Metadata:
    X: degrees
    Y: degrees
    CRS: EPSG:4326
Data Path:
    c:\code\trackio\notebooks\example0

In [3]:
#make metadata mappers
meta_mapper = ds.make_meta_mapper('Length', ncores=4)

meta_mapper

Making meta mappers:   0%|          | 0/172 [00:00<?, ?it/s]

Making meta mappers: 100%|██████████| 172/172 [00:03<00:00, 50.15it/s]


{0.0: None,
 11.0: None,
 12.0: None,
 13.0: None,
 15.0: None,
 16.0: None,
 17.0: None,
 18.0: None,
 19.0: None,
 20.0: None,
 21.0: None,
 22.0: None,
 23.0: None,
 24.0: None,
 25.0: None,
 26.0: None,
 27.0: None,
 28.0: None,
 29.0: None,
 30.0: None,
 31.0: None,
 32.0: None,
 33.0: None,
 34.0: None,
 35.0: None,
 36.0: None,
 37.0: None,
 38.0: None,
 39.0: None,
 42.0: None,
 44.0: None,
 48.0: None,
 52.0: None,
 60.0: None,
 84.0: None,
 88.0: None,
 94.0: None,
 159.0: None,
 169.0: None,
 170.0: None,
 227.0: None,
 228.0: None,
 240.0: None,
 248.0: None,
 333.0: None,
 nan: None}

# Or make multiple at once.

In [4]:
#make metadata mappers
meta_mappers = ds.make_meta_mapper(['Length', 'Width'], ncores=4)

meta_mappers

Making meta mappers: 100%|██████████| 172/172 [00:03<00:00, 51.73it/s]


{'Length': {0.0: None,
  11.0: None,
  12.0: None,
  13.0: None,
  15.0: None,
  16.0: None,
  17.0: None,
  18.0: None,
  19.0: None,
  20.0: None,
  21.0: None,
  22.0: None,
  23.0: None,
  24.0: None,
  25.0: None,
  26.0: None,
  27.0: None,
  28.0: None,
  29.0: None,
  30.0: None,
  31.0: None,
  32.0: None,
  33.0: None,
  34.0: None,
  35.0: None,
  36.0: None,
  37.0: None,
  38.0: None,
  39.0: None,
  42.0: None,
  44.0: None,
  48.0: None,
  52.0: None,
  60.0: None,
  84.0: None,
  88.0: None,
  94.0: None,
  159.0: None,
  169.0: None,
  170.0: None,
  227.0: None,
  228.0: None,
  240.0: None,
  248.0: None,
  333.0: None,
  nan: None},
 'Width': {0.0: None,
  4.0: None,
  5.0: None,
  6.0: None,
  7.0: None,
  8.0: None,
  9.0: None,
  10.0: None,
  11.0: None,
  12.0: None,
  14.0: None,
  18.0: None,
  21.0: None,
  26.0: None,
  27.0: None,
  32.0: None,
  33.0: None,
  37.0: None,
  42.0: None,
  nan: None}}

# Now make up some mapped data.

In [5]:
#descriptor by width
for k in meta_mappers['Width'].keys():
      if k <= 20:
            meta_mappers['Width'][k] = 'narrow'
      else:
            meta_mappers['Width'][k] = 'wide'      
for k in  meta_mappers['Length'].keys():
      if k <= 20:
            meta_mappers['Length'][k] = 'short'
      else:
            meta_mappers['Length'][k] = 'long' 

# Now you can map these to the agents metadata.

In [6]:
#map metadata
ds = ds.map_meta('Width', 
                 'How Wide?', 
                 meta_mappers['Width'],
                 ncores=4,
                 fill='Unknown')

Mapping metadata to agent: 100%|██████████| 172/172 [00:03<00:00, 52.02it/s]


In [7]:
#map multiple metadata at once
ds = ds.map_meta(['Width', 'Length'], 
                 ['How Wide?', 'How Long?'], 
                 meta_mappers,
                 ncores=4,
                 fill='Unknown')

Mapping metadata to agent: 100%|██████████| 172/172 [00:03<00:00, 48.33it/s]


# Now if you refresh the metadata tables, you'll see this data in the `agent.db` file by accessing `Dataset.agents`.

In [8]:
#refresh metadata
ds = ds.refresh_meta(ncores=4)

ds.agents.head()

Refreshing metadata:   0%|          | 0/172 [00:00<?, ?it/s]

Refreshing metadata: 100%|██████████| 172/172 [00:03<00:00, 56.26it/s]  


New meta/databases saved to c:\code\trackio\notebooks\example0


,MMSI,Name,AISCode,Length,Width,How Wide?,How Long?,npoints,ntracks,Xmin,Xmax,Ymin,Ymax,Start Time,End Time,File,geometry
Agent ID,,,,,,,,,,,,,,,,,
Vessel_367459570,367459570,CORAL COAST,31.0,34.0,9.0,narrow,long,59,1,-74.02483,-73.9545,40.698,40.7605,2021-01-01 10:13:36,2021-01-01 11:19:46,c:\code\trackio\notebooks\example0\Vessel_3674...,"POLYGON ((-73.95450 40.69800, -73.95450 40.760..."
Vessel_367796050,367796050,SEA LION,31.0,19.0,6.0,narrow,short,1242,1,-74.07765,-74.07759,40.68621,40.68627,2021-01-01 00:17:17,2021-01-01 23:59:07,c:\code\trackio\notebooks\example0\Vessel_3677...,"POLYGON ((-74.07759 40.68621, -74.07759 40.686..."
Vessel_338862000,338862000,FORT SCHUYLER,31.0,29.0,NaN,Unknown,long,175,1,-74.12821,-74.02601,40.64329,40.70685,2021-01-01 00:17:13,2021-01-01 21:39:36,c:\code\trackio\notebooks\example0\Vessel_3388...,"POLYGON ((-74.02601 40.64329, -74.02601 40.706..."
Vessel_338458000,338458000,NaN,NaN,NaN,NaN,Unknown,Unknown,241,1,-74.12806,-73.95431,40.64077,40.76077,2021-01-01 17:27:11,2021-01-01 21:58:32,c:\code\trackio\notebooks\example0\Vessel_3384...,"POLYGON ((-73.95431 40.64077, -73.95431 40.760..."
Vessel_367597530,367597530,HUNTS POINT,90.0,88.0,21.0,wide,long,1245,1,-73.96945,-73.96941,40.70167,40.70171,2021-01-01 00:17:21,2021-01-01 23:59:01,c:\code\trackio\notebooks\example0\Vessel_3675...,"POLYGON ((-73.96941 40.70167, -73.96941 40.701..."


# You can also do the same thing with any dynamic data field.

In [9]:
#make data mapper
data_mappers = ds.make_data_mapper(['Draft','Speed'],
                                   ncores=4)

#map the draft data to something else
for k in data_mappers['Draft'].keys():
      if k<=5:
            data_mappers['Draft'][k] = 'shallow'
      else:
            data_mappers['Draft'][k] = 'deep'
            
#map the speed data to something else
for k in data_mappers['Speed'].keys():
      if k<=5:
            data_mappers['Speed'][k] = 'slow'
      else:
            data_mappers['Speed'][k] = 'fast'

data_mappers

Making data mappers: 100%|██████████| 172/172 [00:03<00:00, 55.42it/s]


{'Draft': {2.3: 'shallow',
  2.5: 'shallow',
  2.7: 'shallow',
  2.9: 'shallow',
  3.0: 'shallow',
  3.3: 'shallow',
  3.4: 'shallow',
  3.5: 'shallow',
  3.6: 'shallow',
  3.7: 'shallow',
  3.8: 'shallow',
  3.9: 'shallow',
  4.0: 'shallow',
  4.1: 'shallow',
  4.2: 'shallow',
  4.3: 'shallow',
  4.4: 'shallow',
  4.5: 'shallow',
  4.6: 'shallow',
  4.9: 'shallow',
  5.0: 'shallow',
  5.2: 'deep',
  5.5: 'deep',
  5.8: 'deep',
  6.1: 'deep',
  9.4: 'deep',
  9.8: 'deep',
  9.9: 'deep',
  10.1: 'deep',
  12.0: 'deep',
  14.0: 'deep',
  14.5: 'deep',
  14.9: 'deep',
  nan: 'deep'},
 'Speed': {0.0: 'slow',
  0.1: 'slow',
  0.2: 'slow',
  0.3: 'slow',
  0.4: 'slow',
  0.5: 'slow',
  0.6: 'slow',
  0.7: 'slow',
  0.8: 'slow',
  0.9: 'slow',
  1.0: 'slow',
  1.1: 'slow',
  1.2: 'slow',
  1.3: 'slow',
  1.4: 'slow',
  1.5: 'slow',
  1.6: 'slow',
  1.7: 'slow',
  1.8: 'slow',
  1.9: 'slow',
  2.0: 'slow',
  2.1: 'slow',
  2.2: 'slow',
  2.3: 'slow',
  2.4: 'slow',
  2.5: 'slow',
  2.6: 'slow'

In [10]:
#map the data
ds = ds.map_data(['Draft','Speed'],
                 ['Depth Requirement', 'How Fast?'],
                 data_mappers,
                 ncores=4)

#refresh metadata
ds = ds.refresh_meta(ncores=4)

Refreshing metadata: 100%|██████████| 172/172 [00:03<00:00, 57.31it/s]  


New meta/databases saved to c:\code\trackio\notebooks\example0


# Now read a random track to view the mapped dynamic data

In [11]:
#read single track
ds.get_track(ds.tracks.index[29]).head(5)

,Time,X,Y,Draft,Status,Coursing,Speed,Depth Requirement,How Fast?
0,2021-01-01 00:17:18,-74.01489,40.69336,NaN,0.0,241.9,0.1,None,slow
1,2021-01-01 00:18:27,-74.01494,40.69330,NaN,0.0,241.4,0.0,None,slow
2,2021-01-01 00:19:38,-74.01493,40.69331,NaN,0.0,238.8,0.0,None,slow
3,2021-01-01 00:20:48,-74.01489,40.69335,NaN,0.0,240.1,0.0,None,slow
4,2021-01-01 00:21:58,-74.01493,40.69330,NaN,0.0,236.6,0.1,None,slow


# You can also map a dynamic data field to boolean Code columns.

# Lets map the `Coursing` data to quadrants 1, 2, 3 or 4 based on the direction.

In [12]:
#make coursing mapper
coursing_mapper = ds.make_data_mapper('Coursing', ncores=4)

#calculate the quadrant
for key in coursing_mapper.keys():
    coursing_mapper[key] = int((key%360)/90) % 4 + 1

Making data mappers: 100%|██████████| 172/172 [00:03<00:00, 56.64it/s]


# Now, map these integer values to boolean Code columns.

In [13]:
#map quadrants to boolean codes
ds = ds.map_data_to_codes('Coursing',
                          coursing_mapper,
                          ncores=4)

#refresh metadata
ds = ds.refresh_meta(ncores=4)

#read single track
ds.get_track(ds.tracks.index[29]).head(10)

Mapping agent dynamic data to coded boolean arrays: 100%|██████████| 172/172 [00:02<00:00, 61.40it/s]
Refreshing metadata: 100%|██████████| 172/172 [00:03<00:00, 56.74it/s]  


New meta/databases saved to c:\code\trackio\notebooks\example0


,Time,X,Y,Draft,Status,Coursing,Speed,Depth Requirement,How Fast?,Code1,Code2,Code3,Code4
0,2021-01-01 00:17:18,-74.01489,40.69336,NaN,0.0,241.9,0.1,None,slow,False,False,True,False
1,2021-01-01 00:18:27,-74.01494,40.69330,NaN,0.0,241.4,0.0,None,slow,False,False,True,False
2,2021-01-01 00:19:38,-74.01493,40.69331,NaN,0.0,238.8,0.0,None,slow,False,False,True,False
3,2021-01-01 00:20:48,-74.01489,40.69335,NaN,0.0,240.1,0.0,None,slow,False,False,True,False
4,2021-01-01 00:21:58,-74.01493,40.69330,NaN,0.0,236.6,0.1,None,slow,False,False,True,False
5,2021-01-01 00:23:08,-74.01491,40.69331,NaN,0.0,231.5,0.0,None,slow,False,False,True,False
6,2021-01-01 00:24:17,-74.01492,40.69332,NaN,0.0,233.6,0.0,None,slow,False,False,True,False
7,2021-01-01 00:25:28,-74.01492,40.69331,NaN,0.0,231.8,0.0,None,slow,False,False,True,False
8,2021-01-01 00:26:37,-74.01493,40.69335,NaN,0.0,238.1,0.0,None,slow,False,False,True,False
9,2021-01-01 00:27:48,-74.01494,40.69332,NaN,0.0,237.9,0.0,None,slow,False,False,True,False


# Now you can see these codes stored in the `agent.db` and `track.db` metadata tables, making it possible to query by these properties.

In [14]:
#code is true if ANY point along ANY track for agent was True
ds.agents.filter(like='Code')

,AISCode,Code1,Code2,Code3,Code4
Agent ID,,,,,
Vessel_367459570,31.0,True,True,True,True
Vessel_367796050,31.0,True,True,False,True
Vessel_338862000,31.0,True,True,True,True
Vessel_338458000,NaN,True,True,True,True
Vessel_367597530,90.0,True,True,True,True
...,...,...,...,...,...
Vessel_367531640,NaN,True,False,False,True
Vessel_367597240,90.0,True,False,True,True
Vessel_367638180,90.0,True,True,True,True


In [15]:
#code is True if ANY point along track was True
ds.tracks.head(5).filter(like='Code')

,AISCode,Code1,Code2,Code3,Code4
Track ID,,,,,
Vessel_367459570_T0,31.0,True,True,True,True
Vessel_367796050_T0,31.0,True,True,False,True
Vessel_338862000_T0,31.0,True,True,True,True
Vessel_338458000_T0,NaN,True,True,True,True
Vessel_367597530_T0,90.0,True,True,True,True


# Finally, you can also drop metadata and dynamic data columns from the agents/tracks in your `Dataset`. This is useful when you want to reduce the data size and delete data you are not using.

In [16]:
#drop metadata from agents
ds = ds.drop_meta(['How Wide?', 'How Long?'],
                  ncores=4)

#drop dynamic data from tracks
ds = ds.drop_data(['Depth Requirement', 'How Fast?'],
                  ncores=4)

#refresh metadata
ds = ds.refresh_meta(ncores=4)

Refreshing metadata: 100%|██████████| 172/172 [00:02<00:00, 58.31it/s]  


New meta/databases saved to c:\code\trackio\notebooks\example0


# Notice now that this data is not present in the metadata tables, or the actual agent/track.

In [17]:
ds.agents.columns

Index(['MMSI', 'Name', 'AISCode', 'Length', 'Width', 'npoints', 'ntracks',
       'Xmin', 'Xmax', 'Ymin', 'Ymax', 'Start Time', 'End Time', 'Code1',
       'Code2', 'Code3', 'Code4', 'File', 'geometry'],
      dtype='object')

In [18]:
ds.tracks.columns

Index(['MMSI', 'Name', 'AISCode', 'Length', 'Width', 'Agent ID', 'npoints',
       'Xmin', 'Xmax', 'Ymin', 'Ymax', 'Start Time', 'End Time', 'Code1',
       'Code2', 'Code3', 'Code4', 'File', 'Track Length', 'Duration', 'Year',
       'Month', 'Xstart', 'Ystart', 'Xend', 'Yend', 'Effective Distance',
       'Min Temporal Resolution', 'Mean Temporal Resolution',
       'Max Temporal Resolution', 'Min Spatial Resolution',
       'Mean Spatial Resolution', 'Max Spatial Resolution', 'geometry'],
      dtype='object')

In [19]:
#get random agent
agent = ds.get_agent(ds.agents.index[0])

#view agent meta
agent.meta

{'MMSI': 367459570,
 'Name': 'CORAL COAST',
 'AISCode': 31.0,
 'Length': 34.0,
 'Width': 9.0,
 'Agent ID': 'Vessel_367459570'}

In [20]:
#view a single track
agent.tracks['T0'].head(5)

,Time,X,Y,Draft,Status,Coursing,Speed,Code1,Code2,Code3,Code4
0,2021-01-01 10:13:36,-73.95450,40.76050,3.9,0.0,212.8,9.6,False,False,True,False
1,2021-01-01 10:14:44,-73.95667,40.75800,3.9,0.0,215.1,9.6,False,False,True,False
2,2021-01-01 10:15:45,-73.95900,40.75583,3.9,0.0,218.7,9.4,False,False,True,False
3,2021-01-01 10:16:55,-73.96150,40.75350,3.9,0.0,219.3,9.4,False,False,True,False
4,2021-01-01 10:17:56,-73.96350,40.75133,3.9,0.0,214.6,9.4,False,False,True,False
